In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from glob import glob
from sklearn.utils import shuffle
from scipy.fftpack import fft
from sklearn.preprocessing import StandardScaler

In [2]:
def SS(df):
    X = df['activity'].values
    X = X.reshape((len(X), 1))
    X = StandardScaler().fit_transform(X)
    df['activity'] = X
    return(df)

In [3]:
def abs_energy(x):
    """
    Returns the absolute energy of the time series which is the sum over the squared values

    .. math::

        E = \\sum_{i=1,\ldots, n} x_i^2

    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: float
    """
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    return np.dot(x, x)

In [4]:
def FetureExtraction(df,Filename):
 df = SS(df)
 df = df['activity'].resample('60T')
 mean = df.mean()
 std = df.std()
 var = df.var()
 kurtosis = df.apply(pd.DataFrame.kurt)
 skewness = df.apply(pd.DataFrame.skew)
 quantile1 = df.quantile(q=0.01)
 quantile2 = df.quantile(q=0.05)
 quantile3 = df.quantile(q=0.25)
 quantile4 = df.quantile(q=0.75)
 quantile5 = df.quantile(q=0.95)
 quantile6 = df.quantile(q=0.99)
 trimed_Mean = df.apply(stats.trim_mean, .1)
 cofvar = std/ mean
 invcofvar = mean/std
 minimum = df.min()
 maxmimum = df.max()
 median = df.median()
 absenergy = df.apply(abs_energy)
 IQR = df.apply(stats.iqr, interpolation = 'midpoint') 
 frames = [mean,std,var,kurtosis,skewness,quantile1,quantile2,quantile3,quantile4,quantile5,quantile6,trimed_Mean,cofvar,invcofvar,minimum,maxmimum,median,absenergy,IQR]
 final_data = pd.concat(frames,axis=1, sort=False)
 final_data.columns = ['mean','std','var','kurtosis','skewness','quantile1','quantile2','quantile3','quantile4','quantile5','quantile6','trimed_Mean','cofvar','invcofvar','minimum','maxmimum','median','absenergy','IQR']
 #final_data['Class'] = condition_1
 condition = Filename
 final_data.insert(loc=0, column='Id', value=condition)
 return(final_data)

In [5]:
stock_files_condition = sorted(glob('./depresjon-dataset/condition/condition_*.csv'))
stock_files_condition

['./depresjon-dataset/condition\\condition_1.csv',
 './depresjon-dataset/condition\\condition_10.csv',
 './depresjon-dataset/condition\\condition_11.csv',
 './depresjon-dataset/condition\\condition_12.csv',
 './depresjon-dataset/condition\\condition_13.csv',
 './depresjon-dataset/condition\\condition_14.csv',
 './depresjon-dataset/condition\\condition_15.csv',
 './depresjon-dataset/condition\\condition_16.csv',
 './depresjon-dataset/condition\\condition_17.csv',
 './depresjon-dataset/condition\\condition_18.csv',
 './depresjon-dataset/condition\\condition_19.csv',
 './depresjon-dataset/condition\\condition_2.csv',
 './depresjon-dataset/condition\\condition_20.csv',
 './depresjon-dataset/condition\\condition_21.csv',
 './depresjon-dataset/condition\\condition_22.csv',
 './depresjon-dataset/condition\\condition_23.csv',
 './depresjon-dataset/condition\\condition_3.csv',
 './depresjon-dataset/condition\\condition_4.csv',
 './depresjon-dataset/condition\\condition_5.csv',
 './depresjon-dat

In [6]:
li = []
for filename in stock_files_condition:
    #condtion = 'condition_'+ filename
    df = pd.read_csv(filename, parse_dates=["timestamp"], index_col="timestamp")
    li.append(df)

In [7]:
print(li[0])

                           date  activity
timestamp                                
2003-05-07 12:00:00  2003-05-07         0
2003-05-07 12:01:00  2003-05-07       143
2003-05-07 12:02:00  2003-05-07         0
2003-05-07 12:03:00  2003-05-07        20
2003-05-07 12:04:00  2003-05-07       166
...                         ...       ...
2003-05-23 15:19:00  2003-05-23         0
2003-05-23 15:20:00  2003-05-23         0
2003-05-23 15:21:00  2003-05-23         0
2003-05-23 15:22:00  2003-05-23         0
2003-05-23 15:23:00  2003-05-23       533

[23244 rows x 2 columns]


In [8]:
print(len(li[0]))

23244


In [9]:
Final_Data1 =[]
#condtion = 'condition_'
for i, df in zip(range(1,len(li)+1),li):
 condtion = 'condition_'+str(i)
 Final_Data1.append(FetureExtraction(df,condtion))

In [10]:
Final_Data1

[                              Id      mean       std       var   kurtosis  \
 timestamp                                                                   
 2003-05-07 12:00:00  condition_1  0.677568  0.882348  0.778538   1.112264   
 2003-05-07 13:00:00  condition_1  0.467159  0.793051  0.628930   0.466533   
 2003-05-07 14:00:00  condition_1  0.448885  0.924378  0.854475   1.249442   
 2003-05-07 15:00:00  condition_1  0.243852  1.012819  1.025803   8.933094   
 2003-05-07 16:00:00  condition_1  0.310951  0.913359  0.834225   3.080803   
 ...                          ...       ...       ...       ...        ...   
 2003-05-23 11:00:00  condition_1 -0.498829  0.000000  0.000000   0.000000   
 2003-05-23 12:00:00  condition_1 -0.498829  0.000000  0.000000   0.000000   
 2003-05-23 13:00:00  condition_1 -0.498829  0.000000  0.000000   0.000000   
 2003-05-23 14:00:00  condition_1 -0.460130  0.164020  0.026902  26.132998   
 2003-05-23 15:00:00  condition_1 -0.228959  0.711741  0.506575 

In [11]:
len(Final_Data1)

23

In [12]:
final_data1 = pd.concat(Final_Data1,axis=0)
final_data1

,Id,mean,std,var,kurtosis,skewness,quantile1,quantile2,quantile3,quantile4,quantile5,quantile6,trimed_Mean,cofvar,invcofvar,minimum,maxmimum,median,absenergy,IQR
timestamp,,,,,,,,,,,,,,,,,,,,
2003-05-07 12:00:00,condition_1,0.677568,0.882348,0.778538,1.112264,1.046916,-0.498829,-0.442648,0.059582,1.147549,2.159138,3.159287,0.590906,1.302229,0.767914,-0.498829,3.645970,0.426198,73.479591,1.093059
2003-05-07 13:00:00,condition_1,0.467159,0.793051,0.628930,0.466533,0.965757,-0.498829,-0.498829,-0.093175,0.758019,1.934585,2.492928,0.375350,1.697603,0.589066,-0.498829,2.719246,0.349820,50.201127,0.918238
2003-05-07 14:00:00,condition_1,0.448885,0.924378,0.854475,1.249442,1.231372,-0.498829,-0.498829,-0.284121,1.055046,2.436985,3.118993,0.314388,2.059276,0.485608,-0.498829,3.391375,0.139355,62.503887,1.354443
2003-05-07 15:00:00,condition_1,0.243852,1.012819,1.025803,8.933094,2.672737,-0.498829,-0.498829,-0.418207,0.539917,1.705281,4.372039,0.048408,4.153424,0.240765,-0.498829,4.830683,-0.105056,64.090195,0.958973
2003-05-07 16:00:00,condition_1,0.310951,0.913359,0.834225,3.080803,1.665956,-0.498829,-0.498829,-0.390202,0.668063,2.099902,3.371381,0.147417,2.937305,0.340448,-0.498829,3.513581,0.054490,55.020732,1.077783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004-06-10 11:00:00,condition_23,-0.580284,0.000000,0.000000,0.000000,0.000000,-0.580284,-0.580284,-0.580284,-0.580284,-0.580284,-0.580284,-0.580284,-0.000000,-inf,-0.580284,-0.580284,-0.580284,20.203781,0.000000
2004-06-10 12:00:00,condition_23,-0.532894,0.242302,0.058710,40.063490,6.098572,-0.580284,-0.580284,-0.580284,-0.580284,-0.556589,0.508658,-0.580284,-0.454691,-2.199295,-0.580284,1.136808,-0.580284,20.502447,0.000000
2004-06-10 13:00:00,condition_23,1.079410,1.832248,3.357134,-1.778328,0.355536,-0.580284,-0.580284,-0.580284,3.025123,3.613771,3.940084,0.953684,1.697454,0.589118,-0.580284,4.022433,-0.075543,267.978422,3.633809


In [13]:
final_data1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9214 entries, 2003-05-07 12:00:00 to 2004-06-10 15:00:00
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Id           9214 non-null   object 
 1   mean         9214 non-null   float64
 2   std          9213 non-null   float64
 3   var          9213 non-null   float64
 4   kurtosis     9212 non-null   float64
 5   skewness     9212 non-null   float64
 6   quantile1    9214 non-null   float64
 7   quantile2    9214 non-null   float64
 8   quantile3    9214 non-null   float64
 9   quantile4    9214 non-null   float64
 10  quantile5    9214 non-null   float64
 11  quantile6    9214 non-null   float64
 12  trimed_Mean  9214 non-null   float64
 13  cofvar       9213 non-null   float64
 14  invcofvar    9213 non-null   float64
 15  minimum      9214 non-null   float64
 16  maxmimum     9214 non-null   float64
 17  median       9214 non-null   float64
 18  absenergy   

In [14]:
final_data1.insert(loc=1, column='class', value='1')
final_data1.insert(loc=2, column='class_str', value='c1')

In [15]:
final_data1

,Id,class,class_str,mean,std,var,kurtosis,skewness,quantile1,quantile2,...,quantile5,quantile6,trimed_Mean,cofvar,invcofvar,minimum,maxmimum,median,absenergy,IQR
timestamp,,,,,,,,,,,,,,,,,,,,,
2003-05-07 12:00:00,condition_1,1,c1,0.677568,0.882348,0.778538,1.112264,1.046916,-0.498829,-0.442648,...,2.159138,3.159287,0.590906,1.302229,0.767914,-0.498829,3.645970,0.426198,73.479591,1.093059
2003-05-07 13:00:00,condition_1,1,c1,0.467159,0.793051,0.628930,0.466533,0.965757,-0.498829,-0.498829,...,1.934585,2.492928,0.375350,1.697603,0.589066,-0.498829,2.719246,0.349820,50.201127,0.918238
2003-05-07 14:00:00,condition_1,1,c1,0.448885,0.924378,0.854475,1.249442,1.231372,-0.498829,-0.498829,...,2.436985,3.118993,0.314388,2.059276,0.485608,-0.498829,3.391375,0.139355,62.503887,1.354443
2003-05-07 15:00:00,condition_1,1,c1,0.243852,1.012819,1.025803,8.933094,2.672737,-0.498829,-0.498829,...,1.705281,4.372039,0.048408,4.153424,0.240765,-0.498829,4.830683,-0.105056,64.090195,0.958973
2003-05-07 16:00:00,condition_1,1,c1,0.310951,0.913359,0.834225,3.080803,1.665956,-0.498829,-0.498829,...,2.099902,3.371381,0.147417,2.937305,0.340448,-0.498829,3.513581,0.054490,55.020732,1.077783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004-06-10 11:00:00,condition_23,1,c1,-0.580284,0.000000,0.000000,0.000000,0.000000,-0.580284,-0.580284,...,-0.580284,-0.580284,-0.580284,-0.000000,-inf,-0.580284,-0.580284,-0.580284,20.203781,0.000000
2004-06-10 12:00:00,condition_23,1,c1,-0.532894,0.242302,0.058710,40.063490,6.098572,-0.580284,-0.580284,...,-0.556589,0.508658,-0.580284,-0.454691,-2.199295,-0.580284,1.136808,-0.580284,20.502447,0.000000
2004-06-10 13:00:00,condition_23,1,c1,1.079410,1.832248,3.357134,-1.778328,0.355536,-0.580284,-0.580284,...,3.613771,3.940084,0.953684,1.697454,0.589118,-0.580284,4.022433,-0.075543,267.978422,3.633809


In [17]:
stock_files_condition = sorted(glob('./depresjon-dataset/control/control_*.csv'))
stock_files_condition

['./depresjon-dataset/control\\control_1.csv',
 './depresjon-dataset/control\\control_10.csv',
 './depresjon-dataset/control\\control_11.csv',
 './depresjon-dataset/control\\control_12.csv',
 './depresjon-dataset/control\\control_13.csv',
 './depresjon-dataset/control\\control_14.csv',
 './depresjon-dataset/control\\control_15.csv',
 './depresjon-dataset/control\\control_16.csv',
 './depresjon-dataset/control\\control_17.csv',
 './depresjon-dataset/control\\control_18.csv',
 './depresjon-dataset/control\\control_19.csv',
 './depresjon-dataset/control\\control_2.csv',
 './depresjon-dataset/control\\control_20.csv',
 './depresjon-dataset/control\\control_21.csv',
 './depresjon-dataset/control\\control_22.csv',
 './depresjon-dataset/control\\control_23.csv',
 './depresjon-dataset/control\\control_24.csv',
 './depresjon-dataset/control\\control_25.csv',
 './depresjon-dataset/control\\control_26.csv',
 './depresjon-dataset/control\\control_27.csv',
 './depresjon-dataset/control\\control_28.

In [18]:
li2 = []
for filename in stock_files_condition:
    #condtion = 'condition_'+ filename
    df = pd.read_csv(filename, parse_dates=["timestamp"], index_col="timestamp")
    li2.append(df)

In [19]:
li2[6]

,date,activity
timestamp,,
2003-11-25 09:00:00,2003-11-25,6
2003-11-25 09:01:00,2003-11-25,4
2003-11-25 09:02:00,2003-11-25,4
2003-11-25 09:03:00,2003-11-25,4
2003-11-25 09:04:00,2003-11-25,4
...,...,...
2003-12-10 19:58:00,2003-12-10,0
2003-12-10 19:59:00,2003-12-10,3
2003-12-10 20:00:00,2003-12-10,0


In [20]:
len(li2)

32

In [21]:
Final_Data2 =[]
#condtion = 'condition_'
for i, df in zip(range(1,len(li2)+1),li2):
 control = 'control_'+str(i)
 Final_Data2.append(FetureExtraction(df,control))

In [22]:
Final_Data2

[                            Id      mean       std       var   kurtosis  \
 timestamp                                                                 
 2003-03-18 15:00:00  control_1  0.200027  0.853704  0.728811  10.067173   
 2003-03-18 16:00:00  control_1  0.573117  1.574316  2.478471   7.914823   
 2003-03-18 17:00:00  control_1  3.475158  2.189299  4.793028  -1.375163   
 2003-03-18 18:00:00  control_1  0.828884  1.043201  1.088269   2.786854   
 2003-03-18 19:00:00  control_1  1.620177  2.124674  4.514239  -0.819656   
 ...                        ...       ...       ...       ...        ...   
 2003-04-23 08:00:00  control_1 -0.320079  0.064049  0.004102  57.991081   
 2003-04-23 09:00:00  control_1 -0.329861  0.000000  0.000000   0.000000   
 2003-04-23 10:00:00  control_1 -0.290101  0.202917  0.041175  27.278329   
 2003-04-23 11:00:00  control_1 -0.326524  0.019206  0.000369  39.907850   
 2003-04-23 12:00:00  control_1 -0.330803  0.003123  0.000010  11.000000   
 
          

In [23]:
len(Final_Data2)

32

In [24]:
final_data2 = pd.concat(Final_Data2,axis=0)

In [25]:
final_data2

,Id,mean,std,var,kurtosis,skewness,quantile1,quantile2,quantile3,quantile4,quantile5,quantile6,trimed_Mean,cofvar,invcofvar,minimum,maxmimum,median,absenergy,IQR
timestamp,,,,,,,,,,,,,,,,,,,,
2003-03-18 15:00:00,control_1,0.200027,0.853704,0.728811,10.067173,3.188435,-0.340218,-0.312772,-0.196943,0.146572,2.015190,3.566256,-0.007636,4.267949,0.234305,-0.340218,3.712914,-0.070930,45.400478,0.345241
2003-03-18 16:00:00,control_1,0.573117,1.574316,2.478471,7.914823,2.721261,-0.340218,-0.340218,-0.259950,0.377884,3.732938,6.705017,0.190302,2.746934,0.364042,-0.340218,7.493306,-0.001882,165.937605,0.643875
2003-03-18 17:00:00,control_1,3.475158,2.189299,4.793028,-1.375163,0.344297,0.566903,0.812888,1.593996,5.617783,7.016873,7.314126,3.377383,0.629985,1.587339,0.515980,7.741880,2.706536,1007.392081,4.146348
2003-03-18 18:00:00,control_1,0.828884,1.043201,1.088269,2.786854,1.382643,-0.325960,-0.245795,-0.051942,1.427417,2.434313,4.111737,0.700468,1.258562,0.794558,-0.340218,4.869472,0.636815,105.430753,1.487990
2003-03-18 19:00:00,control_1,1.620177,2.124674,4.514239,-0.819656,0.639974,-0.340218,-0.340218,-0.340218,3.267553,5.235428,6.391538,1.372682,1.311384,0.762553,-0.340218,7.016873,0.750744,423.838469,3.756225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2003-12-01 08:00:00,control_32,-0.194495,0.800346,0.640554,17.928978,4.056770,-0.426260,-0.426260,-0.426260,-0.426260,1.489998,3.043879,-0.426260,-4.114987,-0.243014,-0.426260,4.262887,-0.426260,40.062386,0.000000
2003-12-01 09:00:00,control_32,0.114147,1.010018,1.020136,6.488370,2.574614,-0.417758,-0.417758,-0.417758,0.107274,2.405509,3.875044,-0.151876,8.848427,0.113014,-0.417758,4.118344,-0.341235,60.969782,0.531408
2003-12-01 10:00:00,control_32,-0.231127,0.462815,0.214198,5.568872,2.564643,-0.434763,-0.434763,-0.426260,-0.381622,0.895246,1.393580,-0.364794,-2.002428,-0.499394,-0.434763,1.461302,-0.417758,15.842875,0.046764


In [26]:
final_data2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17021 entries, 2003-03-18 15:00:00 to 2003-12-01 12:00:00
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Id           17021 non-null  object 
 1   mean         17016 non-null  float64
 2   std          17016 non-null  float64
 3   var          17016 non-null  float64
 4   kurtosis     17015 non-null  float64
 5   skewness     17016 non-null  float64
 6   quantile1    17016 non-null  float64
 7   quantile2    17016 non-null  float64
 8   quantile3    17016 non-null  float64
 9   quantile4    17016 non-null  float64
 10  quantile5    17016 non-null  float64
 11  quantile6    17016 non-null  float64
 12  trimed_Mean  17016 non-null  float64
 13  cofvar       17016 non-null  float64
 14  invcofvar    17016 non-null  float64
 15  minimum      17016 non-null  float64
 16  maxmimum     17016 non-null  float64
 17  median       17016 non-null  float64
 18  absenergy  

In [27]:
final_data2.insert(loc=1, column='class', value='0')
final_data2.insert(loc=2, column='class_str', value='c0')

In [28]:
final_data2

,Id,class,class_str,mean,std,var,kurtosis,skewness,quantile1,quantile2,...,quantile5,quantile6,trimed_Mean,cofvar,invcofvar,minimum,maxmimum,median,absenergy,IQR
timestamp,,,,,,,,,,,,,,,,,,,,,
2003-03-18 15:00:00,control_1,0,c0,0.200027,0.853704,0.728811,10.067173,3.188435,-0.340218,-0.312772,...,2.015190,3.566256,-0.007636,4.267949,0.234305,-0.340218,3.712914,-0.070930,45.400478,0.345241
2003-03-18 16:00:00,control_1,0,c0,0.573117,1.574316,2.478471,7.914823,2.721261,-0.340218,-0.340218,...,3.732938,6.705017,0.190302,2.746934,0.364042,-0.340218,7.493306,-0.001882,165.937605,0.643875
2003-03-18 17:00:00,control_1,0,c0,3.475158,2.189299,4.793028,-1.375163,0.344297,0.566903,0.812888,...,7.016873,7.314126,3.377383,0.629985,1.587339,0.515980,7.741880,2.706536,1007.392081,4.146348
2003-03-18 18:00:00,control_1,0,c0,0.828884,1.043201,1.088269,2.786854,1.382643,-0.325960,-0.245795,...,2.434313,4.111737,0.700468,1.258562,0.794558,-0.340218,4.869472,0.636815,105.430753,1.487990
2003-03-18 19:00:00,control_1,0,c0,1.620177,2.124674,4.514239,-0.819656,0.639974,-0.340218,-0.340218,...,5.235428,6.391538,1.372682,1.311384,0.762553,-0.340218,7.016873,0.750744,423.838469,3.756225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2003-12-01 08:00:00,control_32,0,c0,-0.194495,0.800346,0.640554,17.928978,4.056770,-0.426260,-0.426260,...,1.489998,3.043879,-0.426260,-4.114987,-0.243014,-0.426260,4.262887,-0.426260,40.062386,0.000000
2003-12-01 09:00:00,control_32,0,c0,0.114147,1.010018,1.020136,6.488370,2.574614,-0.417758,-0.417758,...,2.405509,3.875044,-0.151876,8.848427,0.113014,-0.417758,4.118344,-0.341235,60.969782,0.531408
2003-12-01 10:00:00,control_32,0,c0,-0.231127,0.462815,0.214198,5.568872,2.564643,-0.434763,-0.434763,...,0.895246,1.393580,-0.364794,-2.002428,-0.499394,-0.434763,1.461302,-0.417758,15.842875,0.046764


In [30]:
final = [final_data1, final_data2]
final_data_Frq = pd.concat(final)

In [31]:
final_data_Frq

,Id,class,class_str,mean,std,var,kurtosis,skewness,quantile1,quantile2,...,quantile5,quantile6,trimed_Mean,cofvar,invcofvar,minimum,maxmimum,median,absenergy,IQR
timestamp,,,,,,,,,,,,,,,,,,,,,
2003-05-07 12:00:00,condition_1,1,c1,0.677568,0.882348,0.778538,1.112264,1.046916,-0.498829,-0.442648,...,2.159138,3.159287,0.590906,1.302229,0.767914,-0.498829,3.645970,0.426198,73.479591,1.093059
2003-05-07 13:00:00,condition_1,1,c1,0.467159,0.793051,0.628930,0.466533,0.965757,-0.498829,-0.498829,...,1.934585,2.492928,0.375350,1.697603,0.589066,-0.498829,2.719246,0.349820,50.201127,0.918238
2003-05-07 14:00:00,condition_1,1,c1,0.448885,0.924378,0.854475,1.249442,1.231372,-0.498829,-0.498829,...,2.436985,3.118993,0.314388,2.059276,0.485608,-0.498829,3.391375,0.139355,62.503887,1.354443
2003-05-07 15:00:00,condition_1,1,c1,0.243852,1.012819,1.025803,8.933094,2.672737,-0.498829,-0.498829,...,1.705281,4.372039,0.048408,4.153424,0.240765,-0.498829,4.830683,-0.105056,64.090195,0.958973
2003-05-07 16:00:00,condition_1,1,c1,0.310951,0.913359,0.834225,3.080803,1.665956,-0.498829,-0.498829,...,2.099902,3.371381,0.147417,2.937305,0.340448,-0.498829,3.513581,0.054490,55.020732,1.077783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2003-12-01 08:00:00,control_32,0,c0,-0.194495,0.800346,0.640554,17.928978,4.056770,-0.426260,-0.426260,...,1.489998,3.043879,-0.426260,-4.114987,-0.243014,-0.426260,4.262887,-0.426260,40.062386,0.000000
2003-12-01 09:00:00,control_32,0,c0,0.114147,1.010018,1.020136,6.488370,2.574614,-0.417758,-0.417758,...,2.405509,3.875044,-0.151876,8.848427,0.113014,-0.417758,4.118344,-0.341235,60.969782,0.531408
2003-12-01 10:00:00,control_32,0,c0,-0.231127,0.462815,0.214198,5.568872,2.564643,-0.434763,-0.434763,...,0.895246,1.393580,-0.364794,-2.002428,-0.499394,-0.434763,1.461302,-0.417758,15.842875,0.046764


In [32]:
final_data_Frq.to_csv (r'final_data_Temp.csv', index = False, header=True)